In [3]:
import itertools
import functools
import collections
import sympy as sp
import numpy as np

In [2]:
def hypU(m, n):
    """
    m: array-like of length r
    n: array-like of length k
    """
    m = np.array(m)
    n = np.array(n)

    r = len(m)
    k = len(n)

    total = 0
    for i in range(k):
        for j in range(r):
            total += np.abs(n[i] - m[j])
    return total / 2

def vecU(m):
    """
    m: array-like of length k
    """
    k = len(m)
    m = np.array(m)
    total = 0
    for i in range(1, k):   # i = 1,...,k-1
        for j in range(i):  # j = 0,...,i-1  (i elements total)
            total += np.abs(m[i] - m[j])
    return -total


def Flav(m, n):
    """
    m: array-like of length k
    n: scalar multiplier
    """
    m = np.array(m)
    return (n / 2) * np.sum(np.abs(m))

# Dressing factor PgU
def PgU(v, t):
    # multiplicities of equal entries in v (like Split[Sort[v]])
    counts = [len(list(g)) for _, g in itertools.groupby(sorted(v))]
    n1 = [k for c in counts for k in range(1, c + 1)]
    if not n1:
        return sp.Integer(1)
    return sp.prod(1 / (1 - t**(2*k)) for k in n1)


# Example calculation for (2)–[4] quiver
# (2)–[4] means a gauge node of rank 2 connected to a flavor node of rank 4


gg = 6 

m_syms = [sp.Symbol('m1'), sp.Symbol('m2')]
summed_expr = 0
t = sp.Symbol('t')

for m1 in range(-gg, gg + 1):
    for m2 in range(-gg, m1 + 1):  # m2 <= m1
        m = [m1, m2]
        vec_contrib = vecU(m)
        flav_contrib = Flav(m, 4)
        pg_contrib = PgU(m, t)
        term = t**(2 * int(vec_contrib + flav_contrib)) * pg_contrib
        summed_expr += term

print("Summed expression:", sp.series(summed_expr, t, 0, 2*gg))  # Series expansion around t=0 up to order 5



Summed expression: 1 + 3*t**2 + 9*t**4 + 18*t**6 + 35*t**8 + 57*t**10 + O(t**12)


In [ ]:

def vecU(m):
    """Vector multiplet for U(len(m)):  -∑_{a<b} |m_a - m_b|  (integer)"""
    total = 0
    for i in range(1, len(m)):
        for j in range(i):
            total += abs(m[i] - m[j])
    return -sp.Integer(total)

def hypU(m, n, mult=1):
    """Bifundamental hyper between U(len(m)) and U(len(n)), with multiplicity."""
    s = 0
    for x in n:
        for y in m:
            s += abs(x - y)
    return sp.Rational(mult * s, 2)

def Flav(m, F):
    """F fundamentals on U(len(m))."""
    return sp.Rational(F, 2) * sum(abs(x) for x in m)

def PgU_for_tuple(m, t):
    """Dressing factor P_{U(N)}(t; m) via residual gauge multiplicities."""
    counts = [len(list(g)) for _, g in itertools.groupby(sorted(m))]
    prod = sp.Integer(1)
    for c in counts:
        for k in range(1, c + 1):
            prod *= 1 / (1 - t ** (2 * k))
    return sp.simplify(prod)

def monotone_tuples(rank, B):
    """All nonincreasing integer tuples m1>=...>=mN with mi in [-B, B]."""
    cur = [0] * rank
    def rec(i, prev):
        if i == rank:
            yield tuple(cur); return
        for v in range(-B, prev + 1):
            cur[i] = v
            yield from rec(i + 1, v)
    yield from rec(0, B)

# --------- Quiver model ---------
class Quiver:
    """
    Unitary quiver:
      - add_node(name, rank, flavor=0)
      - add_edge(u, v, mult=1)  # undirected, multiplicity = # of bifundamentals
    """
    def __init__(self):
        self.nodes = {}  # name -> (rank, flavor)
        self.edges = collections.defaultdict(int)  # frozenset({u,v}) -> mult

    def add_node(self, name, rank, flavor=0):
        self.nodes[name] = (rank, flavor)

    def add_edge(self, u, v, mult=1):
        self.edges[frozenset((u, v))] += mult

    def neighbors(self, u):
        res = []
        for e, m in self.edges.items():
            if u in e:
                v = next(iter(e - {u}))
                res.append((v, m))
        return res

    def is_tree(self):
        n, m = len(self.nodes), len(self.edges)
        if n == 0: return True
        if m != n - 1: return False
        start = next(iter(self.nodes))
        seen, stack = {start}, [start]
        while stack:
            x = stack.pop()
            for y, _ in self.neighbors(x):
                if y not in seen:
                    seen.add(y); stack.append(y)
        return len(seen) == n

# --------- Engines ---------
def _hs_bruteforce_exact(Q, B, t):
    names = list(Q.nodes.keys())
    M = {n: list(monotone_tuples(Q.nodes[n][0], B)) for n in names}
    Pg = {n: {m: PgU_for_tuple(m, t) for m in M[n]} for n in names}
    hs = sp.Integer(0)
    for assign in itertools.product(*[M[n] for n in names]):
        m_by = {n: assign[i] for i, n in enumerate(names)}
        dim = sp.Integer(0)
        P = sp.Integer(1)
        for n in names:
            rank, F = Q.nodes[n]
            m = m_by[n]
            dim += vecU(m) + Flav(m, F)
            P *= Pg[n][m]
        for e, mult in Q.edges.items():
            u, v = tuple(e)
            dim += hypU(m_by[u], m_by[v], mult)
        hs += t ** (2 * dim) * P
    return sp.simplify(hs)

def _hs_tree_exact(Q, B, t, root=None):
    if not Q.is_tree():
        raise ValueError("Tree method requires an acyclic quiver.")
    root = root or next(iter(Q.nodes))
    M  = {n: tuple(monotone_tuples(Q.nodes[n][0], B)) for n in Q.nodes}
    Pg = {n: {m: PgU_for_tuple(m, t) for m in M[n]} for n in Q.nodes}

    @functools.lru_cache(None)
    def Z(node, parent, parent_m):
        total = sp.Integer(0)
        for m in M[node]:
            dim = vecU(m) + Flav(m, Q.nodes[node][1])
            if parent is not None:
                mult = Q.edges[frozenset((node, parent))]
                dim += hypU(m, parent_m, mult)
            P = Pg[node][m]
            prod_children = sp.Integer(1)
            for nbr, _ in Q.neighbors(node):
                if nbr == parent: continue
                prod_children *= Z(nbr, node, m)
            total += t ** (2 * dim) * P * prod_children
        return sp.simplify(total)

    hs = sp.Integer(0)
    for m0 in M[root]:
        dim0 = vecU(m0) + Flav(m0, Q.nodes[root][1])
        P0   = Pg[root][m0]
        prod = sp.Integer(1)
        for nbr, _ in Q.neighbors(root):
            prod *= Z(nbr, root, m0)
        hs += t ** (2 * dim0) * P0 * prod
    return sp.simplify(hs)

def hilbert_series(quiver, order=12, cutoff=3, method="auto",
                   stabilize=True, max_cutoff=7, root=None, t=None):
    """
    Compute unrefined HS(t) via the monopole formula.

    Args:
      quiver: Quiver() with U(N) nodes, flavors on nodes, bifundamentals on edges.
      order:  series order (returns up to t^(order-2) with O(t^order)).
      cutoff: initial |m_i| bound (GNO charges in [-cutoff, cutoff]).
      method: 'auto' (tree if possible), 'tree', or 'brute'.
      stabilize: if True, increase cutoff until coefficients up to 'order' stabilize.
      max_cutoff: upper bound for stabilization loop.
      root: root node name for tree DP (optional).
      t: sympy Symbol to use (default: sp.Symbol('t')).

    Returns:
      sympy series in t to the requested order.
    """
    t = t or sp.Symbol('t')
    use_tree = (method == "tree") or (method == "auto" and quiver.is_tree())

    def compute(B):
        expr = _hs_tree_exact(quiver, B, t, root) if use_tree else _hs_bruteforce_exact(quiver, B, t)
        return sp.series(expr, t, 0, order)

    if not stabilize:
        return compute(cutoff)

    prev = None
    B = cutoff
    while B <= max_cutoff:
        cur = compute(B)
        if prev is not None and sp.expand(cur - prev) == 0:
            return sp.series(sp.expand(cur), t, 0, order)
        prev = cur
        B += 1
    # Not fully stabilized; return the last best
    return sp.series(sp.expand(prev), t, 0, order)


In [ ]:
import itertools
import functools
import collections
from typing import Dict, Iterable, Tuple, Optional
import sympy as sp
import networkx as nx

from graph_model import QuiverGraph
from PySide6.QtWidgets import QMessageBox

# ----------------- Monopole-formula primitives (unitary) -----------------

def vecU(m: Tuple[int, ...]) -> sp.Integer:
    """Vector multiplet contribution for U(N):  -∑_{a<b}|m_a - m_b|  (integer)."""
    s = 0
    for i in range(1, len(m)):
        for j in range(i):
            s += abs(m[i] - m[j])
    return sp.Integer(-s)

def hypU(m_left: Tuple[int, ...], m_right: Tuple[int, ...], mult: int = 1) -> sp.Rational:
    """Bifundamental hyper between U(len(left)) and U(len(right)), times multiplicity."""
    s = 0
    for a in m_left:
        for b in m_right:
            s += abs(a - b)
    return sp.Rational(mult * s, 2)

def Flav(m: Tuple[int, ...], F: int) -> sp.Rational:
    """F fundamentals on U(len(m))."""
    return sp.Rational(F, 2) * sum(abs(x) for x in m)

def PgU_for_tuple(m: Tuple[int, ...], t: sp.Symbol) -> sp.Expr:
    """Residual gauge-group dressing P_{U(N)}(t; m)."""
    counts = [len(list(g)) for _, g in itertools.groupby(sorted(m))]
    if not counts:
        return sp.Integer(1)
    prod = sp.Integer(1)
    for c in counts:
        for k in range(1, c + 1):
            prod *= 1 / (1 - t ** (2 * k))
    return sp.simplify(prod)

def monotone_tuples(rank: int, B: int) -> Iterable[Tuple[int, ...]]:
    """
    Generate nonincreasing integer tuples (m1>=...>=mN) with each mi in [-B, B].
    """
    cur = [0] * rank
    def rec(i: int, prev: int):
        if i == rank:
            yield tuple(cur); return
        for v in range(-B, prev + 1):
            cur[i] = v
            yield from rec(i + 1, v)
    yield from rec(0, B)

# ----------------- Convert your QuiverGraph to a compact model -----------------

def _edge_mult_sum(G: nx.MultiGraph, u, v) -> int:
    """Sum 'mult' attributes across parallel edges (default 1 each)."""
    data = G.get_edge_data(u, v, default={})
    total = 0
    for k, attr in data.items():
        total += int(attr.get("mult", 1))
    return total

def sanitize_quiver_graph(G: nx.MultiGraph):
    """
    Build a compact unitary-quiver description from your QuiverGraph.

    Returns:
      gauge_nodes: list of gauge node IDs (order is fixed)
      ranks: dict node -> rank
      Ffund: dict node -> total fundamental count from attached flavor nodes
      bifund: dict frozenset({u,v}) -> multiplicity  (gauge-gauge edges only)
    """
    gauge_nodes = [n for n, a in G.nodes(data=True) if a.get("flav_gauge") == "gauge"]
    flav_nodes  = [n for n, a in G.nodes(data=True) if a.get("flav_gauge") == "flav"]

    # Validate: only unitary gauge groups for now
    for n in gauge_nodes + flav_nodes:
        gp = G.nodes[n].get("gp_type")
        if gp != "U":
            QMessageBox.warning(None, "Non-unitary", f"Only unitary gauge nodes supported; {gp} node detected.")
            return None, None, None, None
    
    

    ranks = {n: int(G.nodes[n]["gp_rank"]) for n in gauge_nodes}

    # Fundamentals per gauge node from attached flavor nodes (rank * edge multiplicity)
    Ffund = {n: 0 for n in gauge_nodes}
    for f in flav_nodes:
        Frank = int(G.nodes[f].get("gp_rank", 0))
        for nbr in G.neighbors(f):
            if nbr in ranks:
                Ffund[nbr] += Frank * _edge_mult_sum(G, f, nbr)

    # Bifundamental multiplicities between gauge nodes
    bifund = {}
    for i, u in enumerate(gauge_nodes):
        for v in gauge_nodes[i + 1:]:
            m = _edge_mult_sum(G, u, v)
            if m:
                bifund[frozenset((u, v))] = m

    return gauge_nodes, ranks, Ffund, bifund

def gauge_is_tree(gauge_nodes, bifund) -> bool:
    """Check if the gauge-only simple graph is a tree."""
    n = len(gauge_nodes)
    if n == 0:
        return True
    m = len(bifund)
    if m != n - 1:
        return False
    # build adjacency
    adj = {u: set() for u in gauge_nodes}
    for e in bifund:
        u, v = tuple(e)
        adj[u].add(v); adj[v].add(u)
    # connectivity
    seen = set()
    stack = [gauge_nodes[0]]
    while stack:
        x = stack.pop()
        if x in seen: continue
        seen.add(x)
        stack.extend(adj[x] - seen)
    return len(seen) == n

# ----------------- Engines (brute & tree-DP) -----------------

def _hs_bruteforce_exact(gauge_nodes, ranks, Ffund, bifund, B: int, t: sp.Symbol) -> sp.Expr:
    Ms = {n: tuple(monotone_tuples(ranks[n], B)) for n in gauge_nodes}
    Pg = {n: {m: PgU_for_tuple(m, t) for m in Ms[n]} for n in gauge_nodes}

    hs = sp.Integer(0)
    for assign in itertools.product(*[Ms[n] for n in gauge_nodes]):
        m = {n: assign[i] for i, n in enumerate(gauge_nodes)}
        dim = sp.Integer(0)
        P   = sp.Integer(1)

        # node contributions
        for n in gauge_nodes:
            dim += vecU(m[n]) + Flav(m[n], Ffund[n])
            P   *= Pg[n][m[n]]

        # edge contributions (bifundamentals)
        for e, mult in bifund.items():
            u, v = tuple(e)
            dim += hypU(m[u], m[v], mult)

        hs += t ** (2 * dim) * P
    return sp.simplify(hs)

def _hs_tree_exact(gauge_nodes, ranks, Ffund, bifund, B: int, t: sp.Symbol) -> sp.Expr:
    if not gauge_nodes:
        return sp.Integer(1)

    # build adjacency with multiplicities
    adj = {u: [] for u in gauge_nodes}
    for e, mult in bifund.items():
        u, v = tuple(e)
        adj[u].append((v, mult))
        adj[v].append((u, mult))

    # Choose root: most central node (highest degree in gauge-only graph)
    degrees = {n: len(adj[n]) for n in gauge_nodes}

    root = max(degrees, key=degrees.get)
    Ms = {n: tuple(monotone_tuples(ranks[n], B)) for n in gauge_nodes}
    Pg = {n: {m: PgU_for_tuple(m, t) for m in Ms[n]} for n in gauge_nodes}

    @functools.lru_cache(None)
    def Z(node, parent, parent_m: Tuple[int, ...]):
        total = sp.Integer(0)
        for m in Ms[node]:
            dim = vecU(m) + Flav(m, Ffund[node])
            if parent is not None:
                mult = {frozenset((node, parent)): mult for (nbr, mult) in adj[node] if nbr == parent}
                # the dict trick above ensures we fetch the correct mult
                mult = next((m for (e, m) in [(frozenset((node, parent)), next((mm for (nb, mm) in adj[node] if nb == parent), 1))]), 1)
            if parent is not None:
                # real multiplicity lookup:
                mlt = next((ml for (nb, ml) in adj[node] if nb == parent), 1)
                dim += hypU(m, parent_m, mlt)

            P = Pg[node][m]
            prod = sp.Integer(1)
            for nb, ml in adj[node]:
                if nb == parent: 
                    continue
                prod *= Z(nb, node, m)
            total += t ** (2 * dim) * P * prod
        return sp.simplify(total)

    hs = sp.Integer(0)
    for m0 in Ms[root]:
        dim0 = vecU(m0) + Flav(m0, Ffund[root])
        P0   = Pg[root][m0]
        prod = sp.Integer(1)
        for nb, ml in adj[root]:
            prod *= Z(nb, root, m0)
        hs += t ** (2 * dim0) * P0 * prod
    return sp.simplify(hs)

# ----------------- Public API -----------------

def hilbert_series_from_quiver_graph(
    G: nx.MultiGraph,
    cutoff: int = 5,
    method: str = "auto",   # 'auto' | 'tree' | 'brute'
    stabilize: bool = True,
    max_cutoff: int = 7,
):
    """
    Unrefined HS(t) from a QuiverGraph built on unitary gauge nodes.

    - Returns a Sympy series in t:  1 + a2 t^2 + ... + O(t^2cutoff)
    - Automatically increases the GNO-charge cutoff until the first 'order'
      coefficients stabilize (unless stabilize=False).

    Notes:
      * Flavor nodes are recognized by node attribute flav_gauge='flav' and only
        contribute fundamentals to adjacent gauge nodes.
      * Edge multiplicities are read from the 'mult' attribute of parallel edges
        (default 1 per edge) and summed.
    """
    t = sp.Symbol('t')

    gauge_nodes, ranks, Ffund, bifund = sanitize_quiver_graph(G)
    if not gauge_nodes:
        return None

    def compute(B: int):
        if method == "brute":
            expr = _hs_bruteforce_exact(gauge_nodes, ranks, Ffund, bifund, B, t)
        else:
            use_tree = (method == "tree") or (method == "auto" and gauge_is_tree(gauge_nodes, bifund))
            if use_tree:
                expr = _hs_tree_exact(gauge_nodes, ranks, Ffund, bifund, B, t)
            else:
                expr = _hs_bruteforce_exact(gauge_nodes, ranks, Ffund, bifund, B, t)
        return sp.series(expr, t, 0, 2*cutoff +1)

    if not stabilize:
        return compute(cutoff)

    prev = None
    B = cutoff
    while B <= max_cutoff:
        cur = compute(B)
        if prev is not None and sp.expand(cur - prev) == 0:
            return sp.series(sp.expand(cur), t, 0, 2*cutoff +1)
        prev = cur
        B += 1
    return sp.series(sp.expand(prev), t, 0, 2*cutoff +1)

In [19]:
from graph_model import QuiverGraph

In [28]:
g = QuiverGraph()
g.add_quiver_node(0, gp_type="U", gp_rank=2, flav_gauge="gauge")
g.add_quiver_node(1, gp_type="U", gp_rank=4, flav_gauge="flavour")
g.add_quiver_edge(0, 1)
HS = hilbert_series_from_quiver_graph(g, cutoff=5)
print(HS)

1 + 3*t**2 + 9*t**4 + 18*t**6 + 35*t**8 + 57*t**10 + O(t**11)


In [29]:
g = QuiverGraph()
g.add_quiver_node("u1_1", gp_type="U", gp_rank=1, flav_gauge="gauge")
g.add_quiver_node("u1_2", gp_type="U", gp_rank=1, flav_gauge="gauge")
g.add_quiver_node("u2", gp_type="U", gp_rank=2, flav_gauge="gauge")
g.add_quiver_node("F", gp_type="U", gp_rank=4, flav_gauge="flavour")
g.add_quiver_edge("u1_1", "u2")
g.add_quiver_edge("u1_2", "u2")
g.add_quiver_edge("F", "u2")

HS = hilbert_series_from_quiver_graph(g, cutoff=5)
print(HS)

1 + 7*t**2 + 35*t**4 + 125*t**6 + 379*t**8 + 984*t**10 + O(t**11)
